*tool - jupyter notebook (python=3.6)

## [감성분석 실습1] Naive Bayes로 스팸메일 필터링
- 새로운 메일이 왔는데 'happy weekend'가 포함되어 있을 경우, 이 메일은 긍정(1)인가, 부정(0)인가?

### 1-1. 직접 구현하기 (by 주한나)

In [1]:
import pandas as pd
import numpy as np

In [2]:
input_file = pd.read_csv('C:/NLP_edu_files/naivebayes_example.csv')
input_file

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [3]:
docs = input_file.copy()

In [4]:
doc_ls = []
for i in range(len(docs)):
     doc_ls.append(input_file.loc[i,'mail'].split())
doc_ls
docs['token'] = doc_ls
docs

,mail,label,token
0,i love you,1,"[i, love, you]"
1,love happy weekend,1,"[love, happy, weekend]"
2,bore work job,0,"[bore, work, job]"
3,i hate you,0,"[i, hate, you]"
4,bore weekend,0,"[bore, weekend]"
5,happy together,1,"[happy, together]"


In [5]:
from collections import defaultdict
word2id = defaultdict(lambda : len(word2id))
for doc in doc_ls:
    for token in doc:
        word2id[token]        # token을 인덱스로 넣자
word2id

defaultdict(<function __main__.<lambda>()>,
            {'i': 0,
             'love': 1,
             'you': 2,
             'happy': 3,
             'weekend': 4,
             'bore': 5,
             'work': 6,
             'job': 7,
             'hate': 8,
             'together': 9})

In [6]:
# docs.loc[docs['label']==1, 'token']

In [7]:
# word2id.keys()

#### 1단계. 각 단어들이 긍정, 부정 분류에 등장한 빈도수를 구한다.

In [8]:
freq = np.zeros((len(word2id), 4))

for token in word2id.keys():
    for x in docs.loc[docs['label']==1, 'token']:
        if token in x:
            freq[word2id[token],0] += 1
            
for token in word2id.keys():
    for x in docs.loc[docs['label']==0, 'token']:
        if token in x:
            freq[word2id[token],1] += 1           
freq

array([[1., 1., 0., 0.],
       [2., 0., 0., 0.],
       [1., 1., 0., 0.],
       [2., 0., 0., 0.],
       [1., 1., 0., 0.],
       [0., 2., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

#### 2단계. 각 단어의 조건부확률 값을 구한다.
- Laplace smoothing 적용하기

In [9]:
import math

In [10]:
for token in word2id.keys():
    i = word2id[token]
    freq[i, 2] = (freq[i,0]+1)/(sum(freq[:,0])+len(word2id))
    freq[i, 3] = (freq[i,1]+1)/(sum(freq[:,1])+len(word2id))
freq

array([[1.        , 1.        , 0.11111111, 0.11111111],
       [2.        , 0.        , 0.16666667, 0.05555556],
       [1.        , 1.        , 0.11111111, 0.11111111],
       [2.        , 0.        , 0.16666667, 0.05555556],
       [1.        , 1.        , 0.11111111, 0.11111111],
       [0.        , 2.        , 0.05555556, 0.16666667],
       [0.        , 1.        , 0.05555556, 0.11111111],
       [0.        , 1.        , 0.05555556, 0.11111111],
       [0.        , 1.        , 0.05555556, 0.11111111],
       [1.        , 0.        , 0.11111111, 0.05555556]])

#### 3단계. 신규텍스트 주어졌을 때 긍/부정일 확률 계산
- Underflow 방지 위한 로그 사용

In [11]:
char = 'happy weekend'
char = char.split()

p_under_p = 0
p_under_n = 0

for token in char:
    i = word2id[token]
    p_under_p = p_under_p + np.log(freq[i,2])
    p_under_n = p_under_n + np.log(freq[i,3])
    
p_under_p = np.exp(p_under_p + np.log(0.5)) # log값을 다시 원래값으로 돌려놓기
p_under_n = np.exp(p_under_n + np.log(0.5))
p_under = p_under_p + p_under_n
p_top = p_under_p
p = p_top/p_under
p

0.7500000000000001

#### 4단계. 함수로 만들어보기

In [12]:
def PosOrNeg(char, num):
    char = char.split()

    p_under_p = 0
    p_under_n = 0

    for token in char:
        i = word2id[token]
        p_under_p = p_under_p + np.log(freq[i,2])
        p_under_n = p_under_n + np.log(freq[i,3])
    p_under_p = np.exp(p_under_p + np.log(0.5))
    p_under_n = np.exp(p_under_n + np.log(0.5))
    p_under = p_under_p + p_under_n
    
    if num == 1:
        p_top = p_under_p
    elif num == 0:
        p_top = p_under_n
    p = p_top/p_under
    
    print("{} 단어들이 {}일 확률은".format(char, '긍정' if num==1 else '부정'),p*100,"% 이다.")
    return (num, p) 

In [13]:
PosOrNeg('happy weekend', 0) 

['happy', 'weekend'] 단어들이 부정일 확률은 24.999999999999996 % 이다.


(0, 0.24999999999999997)

In [14]:
PosOrNeg('happy weekend', 1)

['happy', 'weekend'] 단어들이 긍정일 확률은 75.00000000000001 % 이다.


(1, 0.7500000000000001)

### 1-2. 직접 구현하기 (by 김현진 선생님)

In [15]:
# 다루기 쉽게 array 형태로 바꾸기
training_set = np.array(input_file)
print(training_set)

[['i love you' 1]
 ['love happy weekend' 1]
 ['bore work job' 0]
 ['i hate you' 0]
 ['bore weekend' 0]
 ['happy together' 1]]


#### 문서별 토큰수(토큰 빈도수) 계산
- 확률 계산을 위한 준비

In [16]:
from  collections import defaultdict
wordfreq = defaultdict(lambda : [0,0])
wordfreq['happy']
wordfreq['weekend']
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'happy': [0, 0], 'weekend': [0, 0]})

In [17]:
# 토큰별로 문서 내 빈도수 카운팅
from  collections import defaultdict
wordfreq = defaultdict(lambda : [0,0]) # lambda : 새로운 단어가 추가될 때 default를  [0,0]으로 하겠다. [긍정,부정]

for doc, point in training_set:
    words = doc.split()
    for word in words:
        if point == 1:
            wordfreq[word][0] += 1 # 긍정인 경우 0번째 인덱스에 1 더하기
        else:
            wordfreq[word][1] += 1 # 부정인 경우 1번째 인덱스에 1 더하기
            
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'i': [1, 1],
             'love': [2, 0],
             'you': [1, 1],
             'happy': [2, 0],
             'weekend': [1, 1],
             'bore': [0, 2],
             'work': [0, 1],
             'job': [0, 1],
             'hate': [0, 1],
             'together': [1, 0]})

In [18]:
# 긍정/부정 빈도수 계산
긍정전체토큰수 = []
부정전체토큰수 = []
for key, (cnt1,cnt0) in wordfreq.items():
    긍정전체토큰수.append(int(cnt1))
    부정전체토큰수.append(int(cnt0))
print(긍정전체토큰수)
print(부정전체토큰수)
전체갯수_긍정 = sum(긍정전체토큰수)
전체갯수_부정 = sum(부정전체토큰수)
print(전체갯수_긍정, 전체갯수_부정)

[1, 2, 1, 2, 1, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 1, 2, 1, 1, 1, 0]
8 8


In [19]:
# 긍정/부정 빈도수 계산_2
pos_sum = 0
neg_sum = 0
for token in wordfreq.keys():
    pos_sum = pos_sum + wordfreq[token][0]
    neg_sum = pos_sum + wordfreq[token][0]
a = pos_sum, neg_sum
a

(8, 9)

#### Training : 토큰별 조건부 확률 계산
- Laplace smoothing 적용하기

In [20]:
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'i': [1, 1],
             'love': [2, 0],
             'you': [1, 1],
             'happy': [2, 0],
             'weekend': [1, 1],
             'bore': [0, 2],
             'work': [0, 1],
             'job': [0, 1],
             'hate': [0, 1],
             'together': [1, 0]})

In [21]:
wordprobs = defaultdict(lambda:[0,0])
for key, (cnt1, cnt0) in wordfreq.items():
    wordprobs[key][0] = (cnt1+1)/(전체갯수_긍정 + len(wordfreq))
    wordprobs[key][1] = (cnt0+1)/(전체갯수_부정 + len(wordfreq))
wordprobs   

defaultdict(<function __main__.<lambda>()>,
            {'i': [0.1111111111111111, 0.1111111111111111],
             'love': [0.16666666666666666, 0.05555555555555555],
             'you': [0.1111111111111111, 0.1111111111111111],
             'happy': [0.16666666666666666, 0.05555555555555555],
             'weekend': [0.1111111111111111, 0.1111111111111111],
             'bore': [0.05555555555555555, 0.16666666666666666],
             'work': [0.05555555555555555, 0.1111111111111111],
             'job': [0.05555555555555555, 0.1111111111111111],
             'hate': [0.05555555555555555, 0.1111111111111111],
             'together': [0.1111111111111111, 0.05555555555555555]})

#### Classify : 신규텍스트가 주어졌을 때 확률 계산
- Underflow 방지 위한 로그 사용

In [22]:
import math
doc = "happy weekend"
tokens = doc.split()
tokens

['happy', 'weekend']

In [23]:
# 초기값을 모두 0으로 처리
log_prob1 = log_prob0 = 0.0

# 모든 단어에 대해 반복
for word, (prob1, prob0) in wordprobs.items():
    if word in tokens:
        log_prob1 += math.log(prob1)
        log_prob0 += math.log(prob0)
log_prob1 += math.log(전체갯수_긍정/(전체갯수_긍정+전체갯수_부정))
log_prob0 += math.log(전체갯수_부정/(전체갯수_부정+전체갯수_부정))

prob1 = math.exp(log_prob1)
print('prob1', prob1)
prob0 = math.exp(log_prob0)
print('prob0', prob0)
print('happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정확률')
print("긍정확률 : {}%".format(prob1/(prob1+prob0)*100))
print("부정확률 : {}%".format(prob0/(prob1+prob0)*100))

prob1 0.009259259259259257
prob0 0.0030864197530864183
happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정확률
긍정확률 : 75.00000000000001%
부정확률 : 24.999999999999996%


In [24]:
# 내 것이랑 비교하기
PosOrNeg('happy weekend', 1)

['happy', 'weekend'] 단어들이 긍정일 확률은 75.00000000000001 % 이다.


(1, 0.7500000000000001)

### 2. Using sklearn

In [25]:
import pandas as pd
input_file = pd.read_csv('C:/NLP_edu_files/naivebayes_example.csv')
input_file

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [26]:
# sklearn에 사용가능하도록 list 형태로 변환
X_train = list(input_file['mail'])
Y_train = list(input_file['label'])
print(X_train)
print(Y_train)

['i love you', 'love happy weekend', 'bore work job', 'i hate you', 'bore weekend', 'happy together']
[1, 1, 0, 0, 0, 1]


In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
# CountVectorizer 선언
count_vect = CountVectorizer()
# fit and transform
X_train_counts = count_vect.fit_transform(X_train)
# MultinomialNB 선언 and fit
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [28]:
# 예측
print(clf.predict(count_vect.transform(["happy weekend"]))) 
# 확률 
print(clf.predict_proba(count_vect.transform(["happy weekend"])))

[1]
[[0.25 0.75]]
